In [43]:
import keras.layers as KL
from keras.models import Model
import keras.backend as K
import tensorflow as tf
from keras.utils import np_utils
from keras.utils.vis_utils import plot_model
import numpy as np
import os
%matplotlib inline

In [44]:
from keras.datasets import mnist

In [45]:
def building_block(filters,block):
    
    # block 1 和 2 的判断代码
    # block = 0 副路     block = 1 主路
    if block != 0:
        stride = 1
    else:
        stride = 2
    
    def f(x):
        # 主通路结构
        y = KL.Conv2D(filters=filters,kernel_size=(1,1),strides=stride)(x)
        y = KL.BatchNormalization(axis=3)(y)
        y = KL.Activation('relu')(y)
        
        y = KL.Conv2D(filters=filters,kernel_size=(3,3),padding='same')(x)
        y = KL.BatchNormalization(axis=3)(y)
        y = KL.Activation('relu')(y)
        
        # 副路
        y = KL.Conv2D(filters=4*filters,kernel_size=(1,1))(y)
        y = KL.BatchNormalization(axis=3)(y)
        
        if block == 0:
            shortcut = KL.Conv2D(filters=4*filters,kernel_size=(1,1),strides=stride)(x)
            shortcut = KL.BatchNormalization()(shortcut)
        else:
            shortcut = x
            
        # 主通路和shortcut add
        y = KL.Add()([y,shortcut])
        y = KL.Activation('relu')(y)
        return y
    return f

In [50]:
def ResNet_Extractor(Xtrain,Ytrain,Xtest,Ytest):
    # 头部  TOP
    # customize your top input
    input = KL.Input([28,28,1])
    x = KL.Conv2D(filters=64,kernel_size=(3,3),padding='same')(input)
    x = KL.BatchNormalization(axis=3)(x)
    x = KL.Activation('relu')(x)
    # 分配你的 布局你的stage 和 block关系
    filters = 64
    block = [2,3,3]
    
    for stage,block_num in enumerate(block):
        print('--stage--',stage,'---')
        for block_id in range(block_num):
            print('--block--',block_id,'----')
            x = building_block(filters=filters,block=block_id)(x)
        filters*=2
            
    # 尾部 输出
    x = KL.AveragePooling2D(pool_size=(2,2))(x)
    x = KL.Flatten()(x)
    x = KL.Dense(units=10,activation='softmax')(x)
    
    
    model = Model(inputs=input,outputs=x)
    
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    history = model.fit(Xtrain,Ytrain,epochs=6,batch_size=200,verbose=1,validation_data=(Xtest,Ytest))
    
    model.save('resnetMnist.h5')
    
    return model

In [51]:
def main():
    (Xtrain,Ytrain),(Xtest,Ytest) = mnist.load_data()
    
    Xtrain = Xtrain.reshape(-1,28,28,1)
    Xtest = Xtest.reshape(-1,28,28,1)
    
    Xtrain = Xtrain / 255.0
    Xtest = Xtest / 255.0
    
    Ytrain = np_utils.to_categorical(Ytrain,10)
    Ytest = np_utils.to_categorical(Ytest,10)
    
    ResNet_Extractor(Xtrain,Ytrain,Xtest,Ytest)

main()

--stage-- 0 ---
--block-- 0 ----


ValueError: Operands could not be broadcast together with shapes (28, 28, 256) (14, 14, 256)